### 0. Kết nối các thư viện, ggsheet, khai báo hàm
***

In [1]:
# thư viện data, tính toán
import pandas as pd
import numpy as np
import re
import math

# query sql
import pypyodbc as py

# các thư viện làm việc với ggsheet
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
import gspread_dataframe as gd #update data vào sheet
from oauth2client.service_account import ServiceAccountCredentials # Đọc dữ liệu từ googlesheets
from google.oauth2 import service_account
from df2gspread import df2gspread as d2g
import json
import pygsheets # cấp quyền edit
import pymssql

# thư viện ngày giờ
import time
import datetime as dt
from datetime import datetime, timedelta
from math import radians, cos, sin, sqrt, atan2
from datetime import date
from time import strftime

import seaborn as sns
%matplotlib inline

In [2]:
db = py.connect("Driver={ODBC Driver 17 for SQL Server};"
               "Server=;"
                "Database=;"
                "uid=;pwd=")
print('Connected to SQL server: DWH')

Connected to SQL server: DWH


In [3]:
scope = ["https://spreadsheets.google.com/feeds",'https://www.googleapis.com/auth/spreadsheets',"https://www.googleapis.com/auth/drive.file","https://www.googleapis.com/auth/drive"]
credentials = ServiceAccountCredentials.from_json_keyfile_name(r'C:\Users\Admin\Desktop\Code\jupyter-GGsheet\service_account.json', scope)
client = gspread.authorize(credentials)

In [4]:
wb_id = '1ccn3vGeiYGRr08z8z-sNkyVjL7gGF9jwKz9ibMyovX0'
sheet = client.open_by_key(wb_id)

In [5]:
#Function chuyển sheet thành Data Frame
def to_dataframe(wb_name,sheet_name,data_row,col):
    a=wb_name.worksheet(sheet_name).get_all_values()
    return pd.DataFrame(data=a[data_row:],columns=a[col])

#Funtion update data vào sheet wb_name:tên wb,target_sheet: tên sheet cần update; data:dữ liệu muốn update
def update_data(wb_name,target_sheet,data):
    existing =to_dataframe(wb_name,target_sheet,1,0)
    updated = existing.append(data)
    gd.set_with_dataframe(wb_name.worksheet(target_sheet),updated)

# Function ghi đè dữ liệu lên gg sheet
def write_data(wb_name,ws_name,ws_range,data):
    wb_name.values_clear(ws_range) 
    sheet=wb_name.worksheet(ws_name)
    set_with_dataframe(sheet,data)
    print('Đã ghi dữ liệu lên sheet '+ ws_name)

In [67]:
df = pd.read_excel(r'C:\Users\Admin\Desktop\Project_chip\Data_ tính rule\data.0306.xlsx',sheet_name='Sheet2')

In [68]:
df.columns

Index(['Imei', 'contract', 'open_time', 'last_event', 'event', 'no_signal_48h',
       'no_signal_days', 'Nghi Tháo nguồn', '12VDC', 'FW', 'Nghi Acquy kém',
       'DC_LOSS/PLUG gần nhất', 'DC_LOSS/PLUG trước đó',
       'Tọa độ DC_LOSS/PLUG gần nhất', 'Tọa độ DC_LOSS/PLUG trước đó',
       'event_data'],
      dtype='object')

### 1. Lấy tọa độ, thời gian log chip
***

#### i. Lấy data raw tọa độ, tgian

In [69]:
sub_df = df.loc[:,['event_data']]

In [70]:
s = sub_df['event_data'].str.split('{').apply(pd.Series,1).stack()
s.name = 'split_data'
s. index = s.index.droplevel(-1)

In [71]:
print(s.iloc[1])

'event_data': 'ACC_SLEEP', 'event_type': '4', 'time': '190521095514', 'dc': '0', 'lng': '105.98792666666667', 'timestamp': 1621418114, 'lat': '10.2302275'}, 


In [25]:
sdf = df.join(s)

In [26]:
sdf.columns

Index(['Imei', 'contract', 'open_time', 'last_event', 'event', 'no_signal_48h',
       'no_signal_days', 'Nghi Tháo nguồn', '12VDC', 'FW', 'Nghi Acquy kém',
       'DC_LOSS/PLUG gần nhất', 'DC_LOSS/PLUG trước đó',
       'Tọa độ DC_LOSS/PLUG gần nhất', 'Tọa độ DC_LOSS/PLUG trước đó',
       'event_data', 'split_data'],
      dtype='object')

##### Note: Để ý rules các tag dữ liệu long-lat. event ... để điều chỉnh extract bằng regex ra đúng

In [27]:
def get_longitude(cell):
    x = re.search('lng\': \'(.+?)\',', cell)
    if x is not None:
        return(x.group(1))

In [72]:
def get_latitude(cell):
    x = re.search('lat\': \'(.+?)\'}',cell)
    if x is not None:
        return(x.group(1))

In [29]:
def get_time(cell):
    x = re.search('timestamp\':(.+?),',cell)
    if x is not None:
        try:
            log_time = int(x.group(1))
            return (datetime.utcfromtimestamp(log_time)+timedelta(hours=7)).strftime('%Y-%m-%d %H:%M:%S')
        except:
            return None

In [30]:
def get_event(cell):
    x = re.search('event_data\': \'(.+?)\',',cell)
    if x is not None:
        return(x.group(1))  

In [73]:
sdf['longitude'] = sdf['split_data'].apply(lambda row:get_longitude(row))

In [74]:
sdf['latitude'] = sdf['split_data'].apply(lambda row:get_latitude(row))

In [75]:
sdf['get_time'] = sdf['split_data'].apply(lambda row:get_time(row))

In [76]:
sdf['event_log'] = sdf['split_data'].apply(lambda row:get_event(row))

In [77]:
sdf.columns

Index(['Imei', 'contract', 'open_time', 'last_event', 'event', 'no_signal_48h',
       'no_signal_days', 'Nghi Tháo nguồn', '12VDC', 'FW', 'Nghi Acquy kém',
       'DC_LOSS/PLUG gần nhất', 'DC_LOSS/PLUG trước đó',
       'Tọa độ DC_LOSS/PLUG gần nhất', 'Tọa độ DC_LOSS/PLUG trước đó',
       'event_data', 'split_data', 'longitude', 'latitude', 'get_time',
       'event_log'],
      dtype='object')

In [78]:
sdf

,Imei,contract,open_time,last_event,event,no_signal_48h,no_signal_days,Nghi Tháo nguồn,12VDC,FW,...,DC_LOSS/PLUG gần nhất,DC_LOSS/PLUG trước đó,Tọa độ DC_LOSS/PLUG gần nhất,Tọa độ DC_LOSS/PLUG trước đó,event_data,split_data,longitude,latitude,get_time,event_log
0,A006876,1687462,2021-03-03 17:02:11,2021-05-19 16:55:24,ACC_SLEEP,1,14,0,0,3.0.1604290516,...,2021-05-19 14:17:55:DC_LOSS,2021-05-19 14:14:15:DC_PLUG,"10.244295666666666,105.98059166666667","10.244295666666666,105.98059166666667","[{'event_data': 'ACC_SLEEP', 'event_type': '4'...",[,None,None,None,None
0,A006876,1687462,2021-03-03 17:02:11,2021-05-19 16:55:24,ACC_SLEEP,1,14,0,0,3.0.1604290516,...,2021-05-19 14:17:55:DC_LOSS,2021-05-19 14:14:15:DC_PLUG,"10.244295666666666,105.98059166666667","10.244295666666666,105.98059166666667","[{'event_data': 'ACC_SLEEP', 'event_type': '4'...","'event_data': 'ACC_SLEEP', 'event_type': '4', ...",105.98792666666667,10.2302275,2021-05-19 16:55:14,ACC_SLEEP
0,A006876,1687462,2021-03-03 17:02:11,2021-05-19 16:55:24,ACC_SLEEP,1,14,0,0,3.0.1604290516,...,2021-05-19 14:17:55:DC_LOSS,2021-05-19 14:14:15:DC_PLUG,"10.244295666666666,105.98059166666667","10.244295666666666,105.98059166666667","[{'event_data': 'ACC_SLEEP', 'event_type': '4'...","'event_data': 'ACC_WAKE', 'event_type': '3', '...",105.988015,10.2302115,2021-05-19 16:41:15,ACC_WAKE
0,A006876,1687462,2021-03-03 17:02:11,2021-05-19 16:55:24,ACC_SLEEP,1,14,0,0,3.0.1604290516,...,2021-05-19 14:17:55:DC_LOSS,2021-05-19 14:14:15:DC_PLUG,"10.244295666666666,105.98059166666667","10.244295666666666,105.98059166666667","[{'event_data': 'ACC_SLEEP', 'event_type': '4'...","'event_data': 'ACC_SLEEP', 'event_type': '4', ...",105.988015,10.2302115,2021-05-19 16:35:55,ACC_SLEEP
0,A006876,1687462,2021-03-03 17:02:11,2021-05-19 16:55:24,ACC_SLEEP,1,14,0,0,3.0.1604290516,...,2021-05-19 14:17:55:DC_LOSS,2021-05-19 14:14:15:DC_PLUG,"10.244295666666666,105.98059166666667","10.244295666666666,105.98059166666667","[{'event_data': 'ACC_SLEEP', 'event_type': '4'...","'event_data': 'ACC_WAKE', 'event_type': '3', '...",105.98776166666667,10.229982666666666,2021-05-19 16:25:55,ACC_WAKE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9573,A000606,1747613,2021-04-22 16:12:14,2021-06-03 01:45:21,RTC_WAKE,0,0,0,0,3.0.0,...,NaN,NaN,NaN,NaN,"[{'event_data': 'RTC_WAKE', 'event_type': '5',...","'event_data': 'ACC_WAKE', 'event_type': '3', '...",106.70105,10.734372166666667,2021-05-28 17:54:16,ACC_WAKE
9573,A000606,1747613,2021-04-22 16:12:14,2021-06-03 01:45:21,RTC_WAKE,0,0,0,0,3.0.0,...,NaN,NaN,NaN,NaN,"[{'event_data': 'RTC_WAKE', 'event_type': '5',...","'event_data': 'ACC_SLEEP', 'event_type': '4', ...",106.70105,10.734372166666667,2021-05-28 17:04:16,ACC_SLEEP
9573,A000606,1747613,2021-04-22 16:12:14,2021-06-03 01:45:21,RTC_WAKE,0,0,0,0,3.0.0,...,NaN,NaN,NaN,NaN,"[{'event_data': 'RTC_WAKE', 'event_type': '5',...","'event_data': 'ACC_WAKE', 'event_type': '3', '...",106.701305,10.734043166666666,2021-05-28 16:53:28,ACC_WAKE
9573,A000606,1747613,2021-04-22 16:12:14,2021-06-03 01:45:21,RTC_WAKE,0,0,0,0,3.0.0,...,NaN,NaN,NaN,NaN,"[{'event_data': 'RTC_WAKE', 'event_type': '5',...","'event_data': 'RTC_WAKE', 'event_type': '5', '...",106.701305,10.734043166666666,2021-05-28 16:34:28,RTC_WAKE


#### ii. Clean dữ liệu

In [79]:
coord_chip = sdf[['Imei','contract','open_time','event_log','longitude','latitude','get_time']]

In [80]:
coord_chip.columns

Index(['Imei', 'contract', 'open_time', 'event_log', 'longitude', 'latitude',
       'get_time'],
      dtype='object')

In [81]:
coord_chip.reset_index(drop=True, inplace=True)

In [82]:
coord_chip.rename(columns = {'contract':'pawnid'}, inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [83]:
data_chip = coord_chip.dropna(subset=['pawnid'])

In [84]:
data_chip.columns

Index(['Imei', 'pawnid', 'open_time', 'event_log', 'longitude', 'latitude',
       'get_time'],
      dtype='object')

In [85]:
data_chip['len_pid'] = data_chip['pawnid'].apply(len)

<ipython-input-85-4c92ad4d3e6a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_chip['len_pid'] = data_chip['pawnid'].apply(len)


In [86]:
data_chip['len_pid'].unique()

array([ 7, 19, 17, 18], dtype=int64)

In [87]:
data_chip_clean = data_chip[data_chip['len_pid'] ==7]

In [88]:
data_chip_clean.drop(columns='len_pid',inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [89]:
df2=data_chip_clean

In [90]:
df2 = df2.drop(df2[df2.get_time == None].index)

In [91]:
df2.reset_index(drop=True, inplace=True)

In [92]:
df2.columns

Index(['Imei', 'pawnid', 'open_time', 'event_log', 'longitude', 'latitude',
       'get_time'],
      dtype='object')

In [93]:
df3 = df2.dropna(subset=['get_time','longitude','latitude'])

In [94]:
df3.reset_index(drop=True, inplace=True)

In [95]:
df3['pawnid'] = pd.to_numeric(df3['pawnid'])
df3['longitude'] = pd.to_numeric(df3['longitude'])
df3['latitude'] = pd.to_numeric(df3['latitude'])

<ipython-input-95-447850962fda>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['pawnid'] = pd.to_numeric(df3['pawnid'])
<ipython-input-95-447850962fda>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['longitude'] = pd.to_numeric(df3['longitude'])
<ipython-input-95-447850962fda>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas

In [96]:
df3.columns

Index(['Imei', 'pawnid', 'open_time', 'event_log', 'longitude', 'latitude',
       'get_time'],
      dtype='object')

In [97]:
df3.isna().sum()

Imei         0
pawnid       0
open_time    0
event_log    0
longitude    0
latitude     0
get_time     0
dtype: int64

##### TEST Lấy rule relocate

In [ ]:
df_re = df3[['longitude','latitude']].astype(str).apply(' '.join, axis=1)
df_re.name = 'longlat'

In [ ]:
df_relocate = df3.join(df_re)

In [ ]:
# tách riêng ngày - giờ 
split_time = df_relocate['get_time'].str.split(" ", n=1, expand=True)
split_time.columns = ['date_ym_log','time_log']
df_relocate = pd.concat([df_relocate,split_time], axis=1)

In [ ]:
df_relocate.reset_index(inplace = True)

In [ ]:
#df_relocate2 = df_relocate[df_relocate['row_id'] == 1]

In [ ]:
data_relocate = pd.merge(df_relocate, data_customer2,on='pawnid', how='left')

In [ ]:
data_relocate.columns

In [ ]:
data_relo2 = data_relocate.dropna(subset=['latitude_cus'])

In [ ]:
ad_data = data_relo2['time_log'].str.split(':', n=2, expand = True)
ad_data.columns = ['hour','minute','second']
data_relo2 = pd.concat([data_relo2,ad_data], axis = 1)

In [ ]:
def get_hour_point(row):
    if int(row) < 30:
        return 0
    elif int(row) >= 30:
        return 0.5

In [ ]:
data_relo2['get_time_hour'] = data_relo2['hour'].apply(int) + data_relo2['minute'].apply(lambda row: get_hour_point(row))

In [ ]:
data_relo2['date_hour_group'] = data_relo2['date_ym_log']
data_relo2 = data_relo2.sort_values(by=['pawnid','date_ym_log','time_log'])

In [ ]:
data_relo2.columns

In [ ]:
fn = lambda x: radians(x)

In [ ]:
data_relo2['long_rad'] = data_relo2['longitude'].apply(fn)
data_relo2['lat_rad'] = data_relo2['latitude'].apply(fn)

data_relo2['long_rad_cs'] = data_relo2['longitude_cus'].apply(fn)
data_relo2['lat_rad_cs'] = data_relo2['latitude_cus'].apply(fn)

In [ ]:
def get_distance(lat_cus,long_cus,lat_chip,long_chip):
    R = 6373.0
    dlon = long_chip - long_cus
    dlat = lat_chip - lat_cus

    a = sin(dlat / 2)**2 + cos(lat_cus) * cos(lat_chip) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [ ]:
# define a function for the new column
fn = lambda row:get_distance(row.lat_rad_cs,row.long_rad_cs,row.lat_rad,row.long_rad)

In [ ]:
# get column data with an index
col = data_relo2.apply(fn,axis=1)

In [ ]:
# assign values to column 'distance_km'
data_relo3 = data_relo2.assign(distance_km = col.values)

In [ ]:
data_relo4  = pd.DataFrame(data_relo3[['pawnid','Imei','shop_name','province','distance_km','get_time_hour','date_ym_log','longlat']])

In [ ]:
data_relo4['date-hour'] = data_relo4['date_ym_log'] + '-' + data_relo4['get_time_hour'].apply(str)

In [ ]:
# Chỉ lấy 1 dòng log-event theo ngày giờ
data_relo4 = data_relo4.sort_values(by=['pawnid', 'date-hour'], ascending = True)
data_relo4['row_id'] = data_relo4.groupby(['pawnid','date-hour']).cumcount()+1

In [ ]:
data_relo5 = data_relo4[data_relo4['row_id'] == 1]

In [ ]:
data_relo5.reset_index(drop=True, inplace=True)

In [ ]:
data_relo5.columns

In [ ]:
data_relo5 = data_relo5.sort_values(by=['pawnid','date_ym_log','get_time_hour'], ascending=True)

In [ ]:
data_relo5.reset_index(drop = True,inplace=True)

In [ ]:
#Lấy các khoảng giao giữa ngày
a = data_relo5['date_ym_log'].apply(lambda x: dt.datetime.strptime(x, '%Y-%m-%d'))
b = a.diff().apply(lambda x: x.days)
b.name = 'date_diff'

In [ ]:
#Đo khoảng thời gian giữa các lần log định vị
c = data_relo5.groupby('pawnid')['get_time_hour'].apply(lambda x: x.diff())
c. name = 'hour_diff'

In [ ]:
#So sánh long-lat giữa 2 thời điểm
d = data_relo5['longlat'] == data_relo5['longlat'].shift()
d.name = 'longlat_diff'

In [ ]:
data_relo5 = pd.concat([data_relo5,b,c,d], axis=1)

In [ ]:
def get_duration(row):
    if (row['date_diff'] == 0) and (row['longlat_diff'] == True) and (row['hour_diff'] <=2):
        return row['hour_diff']
    elif (row['date_diff']  == 1) and (row['longlat_diff'] == True) and ((row['hour_diff']+24) <= 2):
        return row['hour_diff']+24
    else:
        return 0

In [ ]:
f = data_relo5.apply(lambda row: get_duration(row), axis=1)
f.name = 'duration'

In [ ]:
data_relo5 = data_relo5.join(f)

In [ ]:
data_relo5.columns

In [ ]:
data_relo6 = data_relo5[['pawnid','Imei','longlat','date_ym_log','get_time_hour','duration','distance_km']]

In [ ]:
data_relo6.to_csv(r'C:\Users\Admin\Desktop\Project_chip\Data_ tính rule\test2.csv')

In [ ]:
data_relo7 = pd.read_csv(r'C:\Users\Admin\Desktop\Project_chip\Data_ tính rule\test2.csv')

In [ ]:
data_relo7

In [ ]:
data_relo8 = data_relo7.groupby(['pawnid','Imei','distance_km','longlat'])['cum_dur'].max()

In [ ]:
data_relo8 = pd.DataFrame(data_relo8)

In [ ]:
data_relo8 = data_relo8.sort_values(by=['pawnid','cum_dur'], ascending=False)
data_relo8['row_id'] = data_relo8.groupby(['pawnid']).cumcount()+1

In [ ]:
data_relo9 = data_relo8[data_relo8['row_id'] <=10 ]

In [ ]:
data_relo9.to_excel(r'C:\Users\Admin\Desktop\Project_chip\Data_ tính rule\test3.xlsx')

In [ ]:
data_relo9

### 2. Lấy tọa độ nhà khách hàng
***

In [6]:
#Lấy tọa độ của nhà KH
get_data ="""
    select shop_name,province,pawnid,
            customerID,codeno, fromdate, long_lat 
    from(
         select shop_name,province,row_number() over (partition by pawnid order by pawnid asc) row_id,
                pawnid,customerID,codeno, fromdate, long_lat 
         from pawn
         left join (select CUSTOMER_WID, long_lat 
                    from W_PAWN_CUSTOEMR_F
                        )cs on cs.CUSTOMER_WID = pawn.CustomerID

         left join W_STORE_SERIES_D sd on sd.OPEN_DT = pawn.shopcode
         where long_lat is not null
         )tab


    where row_id = 1
    """
data = pd.read_sql_query(get_data,db)


In [7]:
data_customer = data

In [8]:
new = data_customer["long_lat"].str.split(",", n=1, expand=True)

In [9]:
new.columns = ['latitude_cus','longitude_cus']

In [10]:
data_customer = pd.concat([data_customer,new], axis=1)

In [11]:
data_customer.columns

Index(['shop_name', 'province', 'pawnid', 'customerid', 'codeno', 'fromdate',
       'long_lat', 'latitude_cus', 'longitude_cus'],
      dtype='object')

In [12]:
def get_numeric(x):
    try:
        return pd.to_numeric(x)
    except:
        return None

In [13]:
data_customer['latitude_cus'] = data_customer['latitude_cus'].apply(get_numeric)
data_customer['longitude_cus'] = data_customer['longitude_cus'].apply(get_numeric)

In [14]:
data_customer2 = data_customer.dropna(subset=['latitude_cus'])

In [15]:
data_customer2 = data_customer2.dropna(subset=['longitude_cus'])

In [16]:
data_customer2.isna().sum()

shop_name        1677
province         1677
pawnid              0
customerid          0
codeno              0
fromdate            0
long_lat            0
latitude_cus        0
longitude_cus       0
dtype: int64

### 3. Merge 2 bảng nhà KH - định vị chip
***

In [98]:
data_chip = pd.merge(df3, data_customer2,on='pawnid', how='left')

In [99]:
data_chip.columns


Index(['Imei', 'pawnid', 'open_time', 'event_log', 'longitude', 'latitude',
       'get_time', 'shop_name', 'province', 'customerid', 'codeno', 'fromdate',
       'long_lat', 'latitude_cus', 'longitude_cus'],
      dtype='object')

### 4. Tính khoảng cách từ vị trí
***

In [100]:
fn = lambda x: radians(x)

In [101]:
data_chip['long_rad'] = data_chip['longitude'].apply(fn)
data_chip['lat_rad'] = data_chip['latitude'].apply(fn)

data_chip['long_rad_cs'] = data_chip['longitude_cus'].apply(fn)
data_chip['lat_rad_cs'] = data_chip['latitude_cus'].apply(fn)

In [102]:
data_chip.columns

Index(['Imei', 'pawnid', 'open_time', 'event_log', 'longitude', 'latitude',
       'get_time', 'shop_name', 'province', 'customerid', 'codeno', 'fromdate',
       'long_lat', 'latitude_cus', 'longitude_cus', 'long_rad', 'lat_rad',
       'long_rad_cs', 'lat_rad_cs'],
      dtype='object')

In [103]:
def get_distance(lat_cus,long_cus,lat_chip,long_chip):
    R = 6373.0
    dlon = long_chip - long_cus
    dlat = lat_chip - lat_cus

    a = sin(dlat / 2)**2 + cos(lat_cus) * cos(lat_chip) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [104]:
# define a function for the new column
fn = lambda row:get_distance(row.lat_rad_cs,row.long_rad_cs,row.lat_rad,row.long_rad)

In [105]:
# get column data with an index
col = data_chip.apply(fn,axis=1)

In [106]:
# assign values to column 'distance_km'
data_chip = data_chip.assign(distance_km = col.values)

### 5. Thống kê số lần xe đi vào bán kính
***

In [107]:
# tách riêng ngày - giờ 
split_time = data_chip['get_time'].str.split(" ", n=1, expand=True)
split_time.columns = ['date_ym_log','time_log']
data_chip = pd.concat([data_chip,split_time], axis=1)

##### Đặt điều kiện trong bán kính nhà khách hàng

In [108]:
def condition(row):
    if row['distance_km'] > 2:
        return 0
    else:
        return 1

In [109]:
data_chip['>2km'] = data_chip.apply(lambda row: condition(row), axis=1)

In [110]:
date_val = pd.to_datetime(data_chip['get_time']).dt.date

In [111]:
data_chip = data_chip.assign(date_log = date_val.values)

In [112]:
data_chip.columns

Index(['Imei', 'pawnid', 'open_time', 'event_log', 'longitude', 'latitude',
       'get_time', 'shop_name', 'province', 'customerid', 'codeno', 'fromdate',
       'long_lat', 'latitude_cus', 'longitude_cus', 'long_rad', 'lat_rad',
       'long_rad_cs', 'lat_rad_cs', 'distance_km', 'date_ym_log', 'time_log',
       '>2km', 'date_log'],
      dtype='object')

In [113]:
data_chip2 = data_chip[['Imei', 'pawnid', 'open_time', 'event_log', 'province',
                        'long_lat', 'date_log','time_log']]

In [ ]:
duplicate_row = data_chip2[data_chip2.duplicated(keep=False)]

In [ ]:
duplicate_row

In [ ]:
duplicate_row.to_excel(r'C:\Users\Admin\Desktop\Project_chip\Data_ tính rule\duplicate_0206.xlsx')

##### Nhóm tín hiệu chip theo ngày - giờ log. Lấy theo đơn vị giờ trong ngày/ 1 ngày = 24 lần

In [115]:
# Chỉ lấy giờ log của con chip
data_chip['date_hour_group'] = data_chip['date_ym_log'] + "-" + data_chip['time_log'].apply(lambda row: row[:2])
data_chip = data_chip.sort_values(by=['pawnid', 'date_log','time_log'], ascending = True)
data_chip['row_id'] = data_chip.groupby(['pawnid','date_hour_group']).cumcount()+1

# Trong 1 giờ chỉ lấy 1 lần log định vị chip
data_chip2 = data_chip[data_chip['row_id'] == 1]

In [116]:
warning_data = data_chip2[['pawnid','codeno','Imei','>2km','long_lat','date_log','time_log','fromdate','event_log','province','shop_name']]

In [117]:
# Chuyển sang dạng bảng kèm điều kiện
df4 = pd.pivot_table(warning_data, values='>2km', index = ['Imei','codeno','pawnid','fromdate','province','shop_name'], columns = 'date_log',aggfunc='sum')

In [118]:
df4.fillna(0, inplace=True)

In [119]:
start_date = pd.to_datetime('2021-05-01').date()

In [120]:
df_result = df4.loc[:,start_date:]

In [121]:
df_result.reset_index(level=['Imei','codeno','pawnid','fromdate','province','shop_name'], inplace=True)

In [122]:
write_data(sheet,'data_03June_2km','data_03June_2km',df_result)

Đã ghi dữ liệu lên sheet data_03June_2km
